In [230]:
import numpy as np
import pandas as pd
import sklearn.utils as utils
import sklearn.linear_model as models
from sklearn.model_selection import cross_validate

In [231]:
def similarity1(o1, o2):
    return np.dot(o1,o2)

In [232]:
def get_data(path, names = None):
    df = pd.read_table(path , header=None, sep=",", names = names)
    data = df.values
    return data

names = ['age', 'W eduction', 'H eduction', 'no of children', 'religon', 'working', 'H of occupation', 'standard of living', 'media expouser', 'cmc']

In [212]:
def get_similarity_Matrix(data, similarity_function):
    #sm = np.empty((len(data),len(data)),dtype=float)
    sm = []
    lables = []
    #lables = np.empty((len(data),len(data)))
    for i in range(len(data)):
        for j in range(i,len(data)): # only get the upper right of the similarity matrix
            sm.append(similarity_function(data[i][:-1],data[j][:-1])) #exculde the label
            lables.append(data[i][-1] == data[j][-1])
           
    return np.array(sm).reshape(-1,1),np.array(lables).reshape(-1,1)

In [216]:
def sample_data(classifer_data):
    classifier_data = classifer_data[classifer_data[:,1].argsort()] #sort by lables
    lable0_count = (classifier_data[:,1] == 0).sum()
    lable1_count = len(classifier_data)- lable0_count
    slice_size = min(lable0_count, lable1_count)    
    # shuffle each lable part individually
    classifier_data[:lable0_count] = utils.shuffle(classifier_data[:lable0_count]) 
    classifier_data[lable0_count:] = utils.shuffle(classifier_data[lable0_count:])
   # sample from each lable by the slice size
    classifier_data0 = classifier_data[:slice_size]
    classifier_data1 = classifier_data[-slice_size:]
    # concatente the samples
    classifier_data = np.concatenate((classifier_data0, classifier_data1), axis= 0)
    return classifier_data

In [223]:
def get_similarity_metrics(data, similarity_fn,  numerical_columns, similarity_type=1): #similarity types 1 - numerical 2 - categorical 3 - mixed
    # numerical columns = no of  numerical cols
    
    get_similarity_Matrix(data, similarity1)
    classifier_data = sample_data(np.concatenate((measures,lables),axis=1))
    classifer_input = classifier_data[:,0].reshape(-1,1)
    classifer_targets = classifier_data[:,1]
    
    
    clf = models.LogisticRegression(max_iter=400)
    #train the classifier
    scoring = ['f1_macro', 'precision_macro', 'recall_macro']
    scores = cross_validate(clf, classifer_input, classifer_targets, cv=5, scoring=scoring, return_train_score=False)
    
    for score in scores:
        if(score == 'fit_time' or score == 'score_time'):
            continue
        print(score, ': ', np.average(scores[score]))
    return

In [224]:
get_similarity_metrics(data, similarity1, 2)

test_f1_macro :  0.5051951658311531
test_precision_macro :  0.5073063947300558
test_recall_macro :  0.5071886991202504
